After I loaded in the data I only did some basic cleaning, enough to start processing the data. 

* Changed the column names to a consistent capitalization.
* Changed Creation Date and Completion Date to datetime fields
* Deleted the columns containing "Dup", so we would only have unique requests
* Dropped the (2) rows in the dataframe which were almost completely NA, including Street Address
* Reset the index


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
url = 'https://data.cityofchicago.org/api/views/7as2-ds3y/rows.csv?accessType=DOWNLOAD'
df = pd.read_csv(url, sep=',', dtype={"Completion Date": object}, low_memory=False)
df.columns = ['Creation Date', 'Status', 'Completion Date', 'Service Request Number', 'Type of Service Request', 'Current Activity', 'Most Recent Action', 'Number of Potholes Filled on Block', 'Street Address', 'Zip', 'X Coordinate', 'Y Coordinate', 'Ward', 'Police District', 'Community Area', 'SSA', 'Latitude', 'Longitude', 'Location']
df['Creation Date'] =  pd.to_datetime(df['Creation Date'])
df['Completion Date'] =  pd.to_datetime(df['Completion Date'])
df = df[~df['Status'].str.contains("Dup")]
df = df.dropna(subset=['Street Address'])
df = df.reset_index(drop=True)
df.head()

,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,Current Activity,Most Recent Action,Number of Potholes Filled on Block,Street Address,Zip,X Coordinate,Y Coordinate,Ward,Police District,Community Area,SSA,Latitude,Longitude,Location
0,2011-01-01,Completed,2011-01-03,11-00002021,Pot Hole in Street,Dispatch Crew,Pothole Patched,5.0,1642 W 99TH ST,60643.0,1.166945e+06,1.839061e+06,19.0,22.0,72.0,NaN,41.714035,-87.665096,"(41.71403466751051, -87.66509581640723)"
1,2011-01-01,Completed,2011-01-03,11-00002273,Pot Hole in Street,Dispatch Crew,Pothole Patched,7.0,3500 S PULASKI RD,60623.0,1.150258e+06,1.881340e+06,22.0,10.0,30.0,NaN,41.830224,-87.724352,"(41.830223541014355, -87.7243519544564)"
2,2011-01-01,Completed,2011-01-03,11-00002650,Pot Hole in Street,Dispatch Crew,Pothole Patched,5.0,7851 S DR MARTIN LUTHER KING JR DR,60619.0,1.180265e+06,1.853038e+06,6.0,6.0,69.0,51.0,41.751434,-87.614820,"(41.75143385051753, -87.61481960290558)"
3,2011-01-01,Completed,2011-01-03,11-00002654,Pot Hole in Street,Dispatch Crew,Pothole Patched,3.0,15 E 79TH ST,60619.0,1.177809e+06,1.852641e+06,6.0,6.0,44.0,NaN,41.750865,-87.623890,"(41.75086472111055, -87.62388985583482)"
4,2011-01-01,Completed,2011-01-03,11-00002656,Pot Hole in Street,Dispatch Crew,Pothole Patched,4.0,359 E 79TH ST,60619.0,1.179971e+06,1.852707e+06,6.0,6.0,44.0,51.0,41.750992,-87.615824,"(41.75099208108744, -87.61582439398472)"


In [80]:
df_potholes = df.copy()
df_potholes['Street Number'] = df_potholes['Street Address'].str.split(' ').str[0]
df_potholes['Street Direction'] = df_potholes['Street Address'].str.split(' ').str[1]
df_potholes['Street Name'] = (df_potholes['Street Address'].str.split(' ').str[2:-1]).str.join(' ')
df_potholes['Street Suffix'] = df_potholes['Street Address'].str.split(' ').str[-1]
df_potholes['Block Number'] = (((df_potholes['Street Number']).astype(int))//100)*100
df_potholes = df_potholes.drop(['Completion Date', 'Type of Service Request', 'Current Activity', 'Most Recent Action'], axis=1)
df_potholes = df_potholes.drop(['Status', 'Zip', 'X Coordinate', 'Y Coordinate', 'Ward', 'Police District', 'Community Area', 'SSA', 'Street Number'], axis=1)
df_potholes['Street Name'] = (df_potholes['Street Address'].str.split(' ').str[2:-1]).str.join(' ')
df_potholes['Street Address'] = df_potholes[['Street Direction', 'Street Name', 'Street Suffix']].apply(lambda x: ' '.join(x), axis=1)
df_potholes = df_potholes.drop(['Street Direction', 'Street Name', 'Street Suffix', 'Location', 'Latitude', 'Longitude'], axis=1)
df_potholes = df_potholes.sort_values('Number of Potholes Filled on Block', ascending=False).drop_duplicates('Service Request Number').sort_index()
df_potholes.head()

,Creation Date,Service Request Number,Number of Potholes Filled on Block,Street Address,Block Number
0,2011-01-01,11-00002021,5.0,W 99TH ST,1600
1,2011-01-01,11-00002273,7.0,S PULASKI RD,3500
2,2011-01-01,11-00002650,5.0,S DR MARTIN LUTHER KING JR DR,7800
3,2011-01-01,11-00002654,3.0,E 79TH ST,0
4,2011-01-01,11-00002656,4.0,E 79TH ST,300


In [81]:
df_mora = pd.read_csv("497_Data/moratoriums.csv")
df_mora = df_mora[df_mora["MORATORIUM TYPE"] == "Street Resurfacing"]
df_mora['START DATE'] = pd.to_datetime(df_mora['START DATE'])
df_mora['EXPIRATION DATE'] = pd.to_datetime(df_mora['EXPIRATION DATE'])
df_mora['STREET NAME SUFFIX'] = df_mora['STREET NAME SUFFIX'].fillna('')
df_mora['Street Address'] = df_mora[['STREET ADDRESS PREFIX', 'STREET NAME', 'STREET NAME SUFFIX']].apply(lambda x: ' '.join(x), axis=1)
df_mora = df_mora.drop_duplicates()
df_mora = df_mora.drop(['STREET ADDRESS PREFIX', 'STREET NAME', 'STREET NAME SUFFIX'], axis=1)
df_mora.head()

,MORATORIUM TYPE,STREET ADDRESS BEGIN,STREET ADDRESS END,START DATE,EXPIRATION DATE,Street Address
0,Street Resurfacing,7100,7400,2017-09-19,2022-09-19,S HONORE ST
1,Street Resurfacing,6100,6130,2017-09-19,2022-09-13,N IONIA AVE
2,Street Resurfacing,4800,5000,2017-09-19,2022-09-19,N CLAREMONT AVE
4,Street Resurfacing,6100,6100,2017-09-19,2022-09-13,N IONIA AVE
5,Street Resurfacing,5890,6000,2017-09-19,2022-09-12,N MARKHAM AVE


In [82]:
df_m2 = df_potholes.merge(df_mora, on='Street Address')
df_m2.head()

,Creation Date,Service Request Number,Number of Potholes Filled on Block,Street Address,Block Number,MORATORIUM TYPE,STREET ADDRESS BEGIN,STREET ADDRESS END,START DATE,EXPIRATION DATE
0,2011-01-01,11-00002021,5.0,W 99TH ST,1600,Street Resurfacing,1732,1820,2018-05-07,2021-10-01
1,2011-01-01,11-00002021,5.0,W 99TH ST,1600,Street Resurfacing,600,630,2018-05-22,2023-04-25
2,2011-01-01,11-00002021,5.0,W 99TH ST,1600,Street Resurfacing,1940,2099,2018-10-25,2023-10-12
3,2011-01-01,11-00002021,5.0,W 99TH ST,1600,Street Resurfacing,1150,2357,2014-09-10,2019-09-09
4,2011-01-01,11-00002021,5.0,W 99TH ST,1600,Street Resurfacing,700,759,2015-07-31,2020-07-31


In [83]:
df_m2['Block Number'] = np.where(((df_m2['Block Number'] >= df_m2['STREET ADDRESS BEGIN']) & (df_m2['Block Number'] <=df_m2['STREET ADDRESS END']) & (df_m2['Creation Date']>df_m2['START DATE'])), df_m2['Block Number'], np.nan)
df_m2 = df_m2.dropna(subset=['Block Number'])
df_m2.head(40)

,Creation Date,Service Request Number,Number of Potholes Filled on Block,Street Address,Block Number,MORATORIUM TYPE,STREET ADDRESS BEGIN,STREET ADDRESS END,START DATE,EXPIRATION DATE
1905,2013-12-23,13-01886611,18.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
1932,2013-12-21,13-01882300,14.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
2274,2014-01-13,14-00056252,2.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
2373,2014-01-14,14-00062059,6.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
2382,2014-01-14,14-00062070,4.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
2769,2014-02-07,14-00181696,4.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
2805,2014-02-06,14-00177117,5.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
3129,2014-03-02,14-00298311,76.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
3489,2014-06-01,14-00820113,35.0,W 99TH ST,1300.0,Street Resurfacing,1230,1300,2013-09-18,2018-09-18
3639,2014-10-01,14-01674563,2.0,W 99TH ST,1600.0,Street Resurfacing,1150,2357,2014-09-10,2019-09-09


In [84]:
df_m2 = df_m2.sort_values(by=['START DATE'], ascending=False)

In [103]:
df_m3 = df_m2.groupby(['Service Request Number', 'Number of Potholes Filled on Block', 'Street Address', 'Creation Date', 'Block Number'])['START DATE', 'STREET ADDRESS BEGIN', 'STREET ADDRESS END', 'EXPIRATION DATE'].first().reset_index()
df_m3 = df_m3[df_m3['Number of Potholes Filled on Block'] > 0.0]
df_m3['SME'] = df_m3['EXPIRATION DATE'].dt.year - df_m3['START DATE'].dt.year
df_m3 = df_m3[df_m3['SME']==5]

86% of the data was 5 year gaps, which makes sense given the CDOT description, but the next highest frequency of (expiration date - start date) (.8%) was 0 year, some rows having only a 10 day gap between start and expiry, which made no sense. Because I had no idea how to interpret that, in the end I weighed that only keeping the 5-year differences data would be better than keeping data I had no idea actually represented resurface date. Also dropped duplicates.

<b>TODO:</b>
Number of potholes by length of time since resurfacing OR resurfacing gap